# INSTALL LIBRARIES

In [1]:
# !pip3 install tensorflow_datasets
# !pip3 install pandas

# IMPORT LIBRARIES

In [ ]:
# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

# General Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

# Neural Network Components
from tensorflow import keras
from keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

warnings.filterwarnings('always')

2025-11-24 08:33:51.266887: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-24 08:33:51.322812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-24 08:33:51.322859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-24 08:33:51.326410: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-24 08:33:51.344830: I tensorflow/core/platform/cpu_feature_guar

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
from ANN_creator_module import NeuralNetworkConstructor

# READING THE DATA SET

In [ ]:
df_path = '/tf/Face_ID1/list_attr_celeba.txt'
df = pd.read_csv(df_path, skiprows=1 ,sep=r'\s+')

In [ ]:
# df.head()

# CREATING THE TENSORFLOW DATASET

In [ ]:
images_dir = '/tf/keras_neural_network/Mis_Tests/img_align_celeba/'
resize_factor = 2
H = 218 // resize_factor
W = 178 // resize_factor
img_size = H  # usar H = W si quieres cuadrado
batch_size = 32
minitrain_size = int(202599*.70)
minival_size = int(202599*.30)

In [ ]:
df.replace(-1, 0, inplace=True)

removed_columns = [
    'Sideburns','Wearing_Earrings','Mouth_Slightly_Open','Heavy_Makeup',
    'Attractive','Rosy_Cheeks','Wearing_Hat','Wearing_Lipstick',
    'Wearing_Necklace','Wearing_Necktie'
]
df.drop(removed_columns, axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
label_cols = df.columns
y = df[label_cols].values.astype(np.float32)
paths = (images_dir + df.index).values

In [ ]:
idx = np.arange(len(paths))
np.random.seed(67)
np.random.shuffle(idx)

train_idx = idx[:minitrain_size]
val_idx = idx[minitrain_size : minitrain_size + minival_size]

train_paths = paths[train_idx]
train_labels = y[train_idx]

val_paths = paths[val_idx]
val_labels = y[val_idx]

In [ ]:
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (img_size, img_size)) / 255.0
    return img, label

In [ ]:
# ====== MINI TRAIN DATASET ======
train_ds = (
    tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
)

# ====== MINI VAL DATASET ======
val_ds = (
    tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size).prefetch(tf.data.AUTOTUNE)
)

# BUILDING THE MODEL

In [ ]:
num_of_classes = 30
inputs = keras.Input((img_size, img_size, 3))
x = inputs
NNC = NeuralNetworkConstructor(
    x, filters=64, total_categories=30, conv_blocks=3, maxpooling_rate=2, 
    downwards_activations=['relu']*6, downwards_dropouts=[0.3]*3, downwards_regularizers=['L2']*6)

conv_outputs = NNC.complete_outputs()
# x = layers.GlobalAveragePooling2D()(conv_outputs)
x = layers.Flatten()(conv_outputs)

x = layers.Dense(
    units=556, 
    activation='relu', 
    name='Dense1_layer')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(
    units=30, 
    activation='softmax', 
    name='clasification_layer')(x)

In [ ]:
from tensorflow.keras.applications import DenseNet121

In [ ]:
base_model = DenseNet121(
    include_top=False,
    input_shape=(H, W, 3),
    input_tensor=inputs, pooling=None)
base_model.trainable = True

In [ ]:
x = inputs
x = base_model(x)
x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(
    units=num_of_classes, 
    activation='sigmoid', 
    name='clasification_layer')(x)

In [ ]:
model = keras.Model(inputs,outputs)
model.summary()

# TRAINING THE MODEL

In [ ]:
model.compile(
    loss="binary_crossentropy", 
    optimizer=eval(f"keras.optimizers.{'RMSprop'}(learning_rate=0.01)"), 
    metrics=["binary_accuracy"]
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    min_delta=0.001,
    patience=10, 
    restore_best_weights=True,
    mode="auto",
    verbose=1,
    baseline=None
)

In [ ]:
# # Load the saved model
# model = keras.models.load_model('my_model.keras')
history = model.fit(
    train_ds,
    epochs=1000, 
    batch_size=batch_size,
    verbose=1,
    validation_data=val_ds,
    callbacks=[early_stopping]
)

In [ ]:
model.save('my_model.keras')

In [ ]:
print(tf.config.list_physical_devices("GPU"))